In [ ]:
# 최종 개선 코드 (solar_api.ipynb 기반 개선 버전)

import pandas as pd
import os
import re
from tqdm import tqdm
from rouge import Rouge
from kiwipiepy import Kiwi  # 형태소 분석기 (Mecab 대신 Kiwi 사용)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Kiwi 초기화
kiwi = Kiwi()

# 환경 설정
config = {
    "data": {
        "path": "./data/",
        "test_file": "test.csv"
    },
    "model": {
        "name": "upstage/SOLAR-10.7B-Instruct-v1.0",  # Instruct 버전 사용
        "temperature": 0.1,
        "max_new_tokens": 150,
        "no_repeat_ngram_size": 3
    },
    "evaluation": {
        "remove_tokens": ['<usr>', '</s>', '<s>', '<pad>']
    },
    "output": {
        "path": "./predictions/",
        "filename": "solar_submission.csv"
    }
}

# 형태소 토큰화 함수
def morpheme_tokenize(text):
    """Kiwi를 이용한 형태소 토큰화"""
    tokens = kiwi.tokenize(text)
    return [token.form for token in tokens]

# 1. 프롬프트 엔지니어링
def build_prompt(dialogue: str) -> str:
    """대화 요약 최적화 프롬프트 생성"""
    prompt = f"""### 대화 요약 ###
다음 대화를 읽고 핵심 내용만 간결하게 3문장 이내로 요약해주세요.
대화 참여자: #Person1#, #Person2# 등

### 대화 내용 ###
{dialogue}

### 요약 ###
"""
    return prompt

# 2. 형태소 분석 기반 ROUGE 계산
def compute_metrics(predictions, references):
    """형태소 분석 기반 ROUGE 점수 계산"""
    rouge = Rouge()
    
    # 형태소 토큰화 및 정제
    processed_preds = []
    processed_refs = []
    
    for pred, ref in zip(predictions, references):
        # 특수 토큰 제거
        pred_clean = ' '.join([token for token in pred.split() 
                              if token not in config["evaluation"]["remove_tokens"]])
        ref_clean = ' '.join([token for token in ref.split() 
                             if token not in config["evaluation"]["remove_tokens"]])
        
        # 형태소 분석
        pred_morphemes = ' '.join(morpheme_tokenize(pred_clean))
        ref_morphemes = ' '.join(morpheme_tokenize(ref_clean))
        
        processed_preds.append(pred_morphemes)
        processed_refs.append(ref_morphemes)
    
    # ROUGE 점수 계산
    scores = rouge.get_scores(processed_preds, processed_refs, avg=True)
    result = {key: value["f"] for key, value in scores.items()}
    
    return result

# 3. Solar 모델 로드
print("모델 로딩 중...")
tokenizer = AutoTokenizer.from_pretrained(config["model"]["name"])
model = AutoModelForCausalLM.from_pretrained(
    config["model"]["name"],
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)
print("모델 로딩 완료!")

# 4. Solar 모델 추론
def generate_summary(dialogue: str) -> str:
    """Solar 모델을 이용한 대화 요약 생성"""
    prompt = build_prompt(dialogue)
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=config["model"]["max_new_tokens"],
            temperature=config["model"]["temperature"],
            no_repeat_ngram_size=config["model"]["no_repeat_ngram_size"],
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # 응답에서 요약 부분 추출
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # 프롬프트 이후 부분만 추출
    summary = full_response.split("### 요약 ###")[-1].strip()
    
    return summary

# 5. 배치 처리 및 결과 저장
def batch_processing(data: pd.DataFrame):
    """데이터 배치 처리"""
    results = []
    
    for idx, row in tqdm(data.iterrows(), total=len(data), desc="요약 생성"):
        dialogue = row["dialogue"]
        summary = generate_summary(dialogue)
        results.append(summary)
        
    return results

# 6. 메인 실행 함수
def main():
    # 데이터 로드
    test_path = os.path.join(config["data"]["path"], config["data"]["test_file"])
    test_data = pd.read_csv(test_path)
    
    print(f"테스트 데이터 수: {len(test_data)}")
    
    # 요약 생성
    summaries = batch_processing(test_data)
    
    # 결과 저장
    os.makedirs(config["output"]["path"], exist_ok=True)
    output_path = os.path.join(config["output"]["path"], config["output"]["filename"])
    
    submission = pd.DataFrame({
        "fname": test_data["fname"],
        "summary": summaries
    })
    
    submission.to_csv(output_path, index=False)
    print(f"결과 저장 완료: {output_path}")
    
    return submission

if __name__ == "__main__":
    main()

모델 로딩 중...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

모델 로딩 완료!
테스트 데이터 수: 499


요약 생성:   0%|          | 0/499 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
요약 생성:   1%|          | 3/499 [07:08<19:41:03, 142.87s/i

KeyboardInterrupt: 

: 